In [ ]:
import time
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json

# EDA

In [ ]:
BASE_PATH = "../data/"

In [ ]:
df = pd.read_csv(BASE_PATH + "dtm/20250504_pre_logic_point_data.csv")

In [ ]:
df.groupby("main_expenses").agg(
    {"budget_project_id": "count", "plan_project": "sum"}
).sort_values("budget_project_id", ascending=False)

In [ ]:
df["len_project_summary"] = df["project_summary"].apply(lambda x: len(x))
df["len_output"] = df["output"].apply(lambda x: len(str(x)))
df["len_outcomes"] = df["outcomes"].apply(lambda x: len(x))

df["input_text_length"] = (
    df["len_project_summary"] + df["len_output"] + df["len_outcomes"]
)

In [ ]:
df.groupby("plan_project").agg(
    {
        "len_project_summary": "mean",
        "len_output": "mean",
        "len_outcomes": "mean",
        "input_text_length": "mean",
    }
)

In [ ]:
df.groupby("plan_project").agg(
    {
        "len_project_summary": "median",
        "len_output": "median",
        "len_outcomes": "median",
        "input_text_length": "median",
    }
)

In [ ]:
df.groupby("plan_project").agg(
    {
        "len_project_summary": "std",
        "len_output": "std",
        "len_outcomes": "std",
        "input_text_length": "std",
    }
)

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300, figsize=(5, 3))
ax = sns.boxplot(
    data=df,
    x="plan_project",
    y="input_text_length",
    color=".8",
    flierprops={"marker": "x"},
)
ax.set_xlabel(r"$\it{Plan}$ project")
ax.set_ylabel("Input text length")
ax.set_ylim(0, 1500)
plt.show()

In [ ]:
pivot_df = df.pivot_table(
    index="logic1_point",
    columns="plan_project",
    values=["budget_project_id"],
    aggfunc="count",
)
pivot_df["percent_0"] = (
    pivot_df[("budget_project_id", 0)] / pivot_df[("budget_project_id", 0)].sum() * 100
)
pivot_df["percent_1"] = (
    pivot_df[("budget_project_id", 1)] / pivot_df[("budget_project_id", 1)].sum() * 100
)
pivot_df

In [ ]:
df.groupby("plan_project").agg({"logic1_point": "mean"})

# regression

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
formula = 'logic1_point ~ plan_project'
result = smf.ols(formula, df.dropna(subset=["logic1_point"])).fit()
print(result.summary())

In [ ]:
formula = 'logic1_point ~ plan_project +  main_expenses'
result = smf.ols(formula, df.dropna(subset=["logic1_point"])).fit()
print(result.summary())

In [ ]:
formula = 'logic1_point ~ plan_project + input_text_length + main_expenses'
result = smf.ols(formula, df.dropna(subset=["logic1_point"])).fit()
print(result.summary())

In [ ]:
formula = 'logic1_point ~ plan_project*main_expenses + input_text_length'
result = smf.ols(formula, df.dropna(subset=["logic1_point"])).fit()
print(result.summary())

In [ ]:
formula = 'logic1_point ~ plan_project + input_text_length'
result = smf.ols(formula, df.dropna(subset=["logic1_point"])).fit()
print(result.summary())

# Hierarchical Bayes

In [ ]:
import arviz as az
import pymc as pm

RANDOM_SEED = 0
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [ ]:
data = df.dropna(subset=["logic1_point"])
# One-Hot Encoding: main_expenses カテゴリカル変数をエンコード
data_encoded = pd.get_dummies(data["main_expenses"], prefix="main_expenses")
data = pd.concat([data, data_encoded], axis=1)

# logic1_pointの標準偏差を計算
logic1_std = data["logic1_point"].std()

with pm.Model() as hierarchical_model:
    # 共有される事前分布 (prior)
    mu_intercept = pm.Normal("mu_intercept", mu=0, sigma=5)  # 切片の平均
    sigma_intercept = pm.HalfNormal("sigma_intercept", sigma=5)  # 切片の標準偏差
    intercept = pm.Normal(
        "intercept",
        mu=mu_intercept,
        sigma=sigma_intercept,
        shape=(data_encoded.shape[1],),
        dims="main_expenses",
    )  # 各 `main_expenses` ごとの切片

    # 観測誤差の標準偏差（sigma）
    sigma = pm.HalfNormal("sigma", sigma=logic1_std)  # 観測誤差の標準偏差

    # `plan_project` の係数の階層ベイズ
    mu_coef_plan = pm.Normal(
        "mu_coef_plan", mu=0, sigma=5
    )  # `plan_project` の係数の共通の事前分布
    sigma_coef_plan = pm.HalfNormal(
        "sigma_coef_plan", sigma=5
    )  # `plan_project` の係数の標準偏差
    coef_plan_project = pm.Normal(
        "coef_plan_project",
        mu=mu_coef_plan,
        sigma=sigma_coef_plan,
        shape=(data_encoded.shape[1],),
        dims="main_expenses",
    )  # 各 `main_expenses` ごとの係数

    # 各調整変数に対する回帰係数
    coef_input_text_length = pm.Normal("coef_input_text_length", mu=0, sigma=5)

    # 線形回帰モデルの式 (階層ベイズ)
    mu = 0
    # 各 `main_expenses` ごとに異なる切片と影響を加える
    for i, expense in enumerate(data_encoded.columns):
        # `main_expenses` ごとの切片を加える
        mu += intercept[i] * data[expense]

        # `plan_project` の係数を加える
        mu += coef_plan_project[i] * data[expense] * data["plan_project"]

    # 調整変数を加味する
    mu += coef_input_text_length * data["input_text_length"]

    # 観測モデル
    observed = pm.Normal("observed", mu=mu, sigma=sigma, observed=data["logic1_point"])

    # サンプリング
    trace_h = pm.sample(500, tune=500, target_accept=0.9, cores=4)

# 結果のサマリー表示
pm.summary(trace_h)

In [ ]:
labels = [
    ": Energy Policy Expenditure",
    ": Small and Medium Enterprise Support Expenditure",
    ": Public Health and Sanitation Expenditure",
    ": Educational Promotion Grants",
    ": Social Welfare and Living Assistance Expenditure",
    ": Science and Technology Promotion Expenditure",
    ": Defense Expenditure",
    ": Employment and Workers' Compensation Expenditure",
    ": Food Security and Supply Expenditure",
]
from_str = [
    "[0]",
    "[1]",
    "[2]",
    "[3]",
    "[4]",
    "[5]",
    "[6]",
    "[7]",
    "[8]",
]


summary_table = pm.summary(trace_h)

# 変換用の辞書を作成
replace_dict = dict(zip(from_str, labels))
coef_list = list(summary_table.index)
# replace()を使って簡単に置換
translated_list = []  # 初期化
for i, coef in enumerate(coef_list):
    for key, value in replace_dict.items():
        coef_list[i] = coef_list[i].replace(key, value)

# 結果を表示
# summary_table.index = coef_list
summary_table[["mean", "sd", "ess_bulk", "ess_tail", "r_hat"]].to_latex()

In [ ]:
# TraceからR-hatを計算
rhat = az.rhat(trace_h)
print(rhat)

In [ ]:
summaey_table = az.summary(trace_h, round_to=3, var_names="coef_plan_project")
summaey_table.index = [
    "Energy Policy Expenditure",
    "Small and Medium Enterprise Support Expenditure",
    "Public Health and Sanitation Expenditure",
    "Educational Promotion Grants",
    "Social Welfare and Living Assistance Expenditure",
    "Science and Technology Promotion Expenditure",
    "Defense Expenditure",
    "Employment and Workers' Compensation Expenditure",
    "Food Security and Supply Expenditure",
]
summaey_table

In [ ]:
pm.model_to_graphviz(hierarchical_model)

In [ ]:
# マッピングするラベルのリスト
labels = [
    "Energy Policy Expenditure",
    "Small and Medium Enterprise Support Expenditure",
    "Public Health and Sanitation Expenditure",
    "Educational Promotion Grants",
    "Social Welfare and Living Assistance Expenditure",
    "Science and Technology Promotion Expenditure",
    "Defense Expenditure",
    "Employment and Workers' Compensation Expenditure",
    "Food Security and Supply Expenditure",
]

# traceからcoef_plan_projectの事後分布をプロット
ax = az.plot_forest(
    trace_h,
    kind="forestplot",
    var_names="coef_plan_project",
    combined=True,
    filter_vars="regex",
    figsize=(12, 7),
)
ax[0].axvline(0, color="black", ls="--")
# y軸ラベルをカスタムラベルに設定（逆順に表示）
ax[0].set_yticklabels(labels[::-1])  # ラベルを逆順に設定

# プロットを表示
plt.show()

In [ ]:
# マッピングするラベルのリスト
labels = [
    "Energy Policy Expenditure",
    "Small and Medium Enterprise Support Expenditure",
    "Public Health and Sanitation Expenditure",
    "Educational Promotion Grants",
    "Social Welfare and Living Assistance Expenditure",
    "Science and Technology Promotion Expenditure",
    "Defense Expenditure",
    "Employment and Workers' Compensation Expenditure",
    "Food Security and Supply Expenditure",
]

# traceからcoef_plan_projectの事後分布をプロット
ax = az.plot_forest(
    trace_h,
    kind="ridgeplot",
    var_names="coef_plan_project",
    combined=True,
    ridgeplot_truncate=False,
    ridgeplot_quantiles=[0.25, 0.5, 0.75],
    ridgeplot_overlap=0.7,
    colors="white",
    figsize=(12, 7),
)
ax[0].axvline(0, color="black", ls="--")
# y軸ラベルをカスタムラベルに設定（逆順に表示）
ax[0].set_yticklabels(labels[::-1])  # ラベルを逆順に設定

# プロットを表示
plt.show()

In [ ]:
ax = az.plot_trace(trace_h, combined=True)

In [ ]:
ax = az.plot_trace(trace_h, combined=True, var_names=["coef_plan_project"])

In [ ]:
with hierarchical_model:
    pm.compute_log_likelihood(trace_h)

In [ ]:
hierarchical_waic = az.waic(trace_h)
hierarchical_waic

# Comparison models

In [ ]:
with pm.Model() as pooled_model:
    # 共有される事前分布 (prior)
    mu_intercept = pm.Normal("mu_intercept", mu=0, sigma=5)  # 切片の平均
    sigma_intercept = pm.HalfNormal("sigma_intercept", sigma=5)  # 切片の標準偏差
    intercept = pm.Normal(
        "intercept",
        mu=mu_intercept,
        sigma=sigma_intercept,
        shape=(data_encoded.shape[1],),
        dims="main_expenses",
    )  # 各 `main_expenses` ごとの切片

    # 観測誤差の標準偏差（sigma）
    sigma = pm.HalfNormal("sigma", sigma=logic1_std)  # 観測誤差の標準偏差

    # `plan_project` の係数の階層ベイズ
    coef_plan_project = pm.Normal(
        "coef_plan_project", mu=0, sigma=5
    )  # 各 `main_expenses` ごとの係数

    # 各調整変数に対する回帰係数
    coef_input_text_length = pm.Normal("coef_input_text_length", mu=0, sigma=5)
    # coef_len_project_summary = pm.Normal('coef_len_project_summary', mu=0, sigma=5)
    # coef_len_output = pm.Normal('coef_len_output', mu=0, sigma=5)
    # coef_len_outcomes = pm.Normal('coef_len_outcomes', mu=0, sigma=5)

    # 線形回帰モデルの式 (階層ベイズ)
    mu = 0
    # 各 `main_expenses` ごとに異なる切片と影響を加える
    for i, expense in enumerate(data_encoded.columns):
        # `main_expenses` ごとの切片を加える
        mu += intercept[i] * data[expense]

    # `plan_project` の係数を加える
    mu += coef_plan_project * data["plan_project"]

    # 調整変数を加味する
    mu += coef_input_text_length * data["input_text_length"]
    # mu += coef_len_project_summary * data['len_project_summary']
    # mu += coef_len_output * data['len_output']
    # mu += coef_len_outcomes * data['len_outcomes']

    # 観測モデル
    observed = pm.Normal("observed", mu=mu, sigma=sigma, observed=data["logic1_point"])

    # サンプリング
    trace_p = pm.sample(500, tune=500, target_accept=0.9, cores=4)

# 結果のサマリー表示
pm.summary(trace_p)

In [ ]:
with pooled_model:
    pm.compute_log_likelihood(trace_p)

In [ ]:
with pm.Model() as naive_model:
    # 共有される事前分布 (prior)
    mu_intercept = pm.Normal("mu_intercept", mu=0, sigma=5)  # 切片の平均
    sigma_intercept = pm.HalfNormal("sigma_intercept", sigma=5)  # 切片の標準偏差
    intercept = pm.Normal(
        "intercept",
        mu=mu_intercept,
        sigma=sigma_intercept,
        shape=(data_encoded.shape[1],),
        dims="main_expenses",
    )  # 各 `main_expenses` ごとの切片

    # 観測誤差の標準偏差（sigma）
    sigma = pm.HalfNormal("sigma", sigma=logic1_std)  # 観測誤差の標準偏差

    # `plan_project` の係数の階層ベイズ
    coef_plan_project = pm.Normal(
        "coef_plan_project", mu=0, sigma=5
    )  # 各 `main_expenses` ごとの係数

    # 線形回帰モデルの式 (階層ベイズ)
    mu = 0
    # 各 `main_expenses` ごとに異なる切片と影響を加える
    for i, expense in enumerate(data_encoded.columns):
        # `main_expenses` ごとの切片を加える
        mu += intercept[i] * data[expense]

    # `plan_project` の係数を加える
    mu += coef_plan_project * data["plan_project"]

    # 観測モデル
    observed = pm.Normal("observed", mu=mu, sigma=sigma, observed=data["logic1_point"])

    # サンプリング
    trace_n = pm.sample(500, tune=500, target_accept=0.9, cores=4)

# 結果のサマリー表示
pm.summary(trace_n)

In [ ]:
with naive_model:
    pm.compute_log_likelihood(trace_n)

In [ ]:
pooled_waic = az.waic(trace_p)
pooled_waic

In [ ]:
df_comp_loo = az.compare({"Random Intercept and Slope Model": trace_h, "Random Intercept Model": trace_p, "Random Intercept Model without covariate": trace_n})
df_comp_loo

In [ ]:
fig, ax = plt.subplots(1,1,dpi = 300,figsize=(10,3))
az.plot_compare(df_comp_loo, insample_dev=False, ax=ax)
ax.set_title("")
plt.show()

In [ ]:
pm.plot_posterior(trace_h, var_names=["coef_plan_project"])

In [ ]:
with hierarchical_model:
    pm.sample_posterior_predictive(trace_h, extend_inferencedata=True, random_seed=rng)

In [ ]:
az.plot_ppc(trace_h, num_pp_samples=100)

# Check sample cases

In [ ]:
df.query(
    "main_expenses in ['エネルギー対策費'] and plan_project == 1 and logic1_point > 4"
).sort_values("budget_project_id", ascending=True)[
    [
        "budget_project_id",
        "project_name",
        "logic1_reason",
        "logic1_point",
        "output",
        "outcomes",
    ]
]

In [ ]:
df.query(
    "main_expenses in ['食料安定供給関係費'] and plan_project == 1 and logic1_point > 4"
).sort_values("budget_project_id", ascending=False)[
    [
        "budget_project_id",
        "project_name",
        "logic1_reason",
        "logic1_point",
        "output",
        "outcomes",
    ]
]

In [ ]:
df.query("plan_project == 1 and logic1_point > 4").sort_values(
    ["main_expenses", "budget_project_id"], ascending=True
)[
    [
        "main_expenses",
        "budget_project_id",
        "project_name",
        "logic1_reason",
        "logic1_point",
        "output",
        "outcomes",
    ]
]